# SC Optimization を LLMでサポート

>  Use OpenAI, LangChain and LM Studio to develop LLMs for SC Optimization

In [ ]:
#| default_exp llm

## Tools

- LM Studio: https://lmstudio.ai/  (download and install)  downlad "mistralai_mistral-7b-instruct-v0.1 or 0.2" and start server! See https://note.com/mikiokubo/n/nb61e5b558ef8 for more detail
- install "openai" package and "langchain" (and new version "langchain-openai") package
<!-- - autogen: https://microsoft.github.io/autogen/docs/Examples/
- autogen-studioも同じgithubにある； agentにskillを追加可能
- OptiGuide: https://github.com/microsoft/OptiGuide -->

In [ ]:
from IPython.display import display, HTML

## LangGraph

何故か登録した関数を呼んでくれない．

動かすにはLangSmith https://smith.langchain.com/ へ登録してAPIキーを環境変数に保管する必要がある．

In [ ]:
import os
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [ ]:
# class MultiplierInput(BaseModel):
#     a: int = Field(description="First number")
#     b: int = Field(description="Second number")


# def multiply(a: int, b: int) -> int:
#     return a * b


# multiplier = StructuredTool.from_function(
#     func=multiply,
#     name="Multiplier",
#     description="Multiply two numbers",
#     args_schema=MultiplierInput,
#     return_direct=False,
# )

In [ ]:
# class AdderInput(BaseModel):
#     a: int = Field(description="First number")
#     b: int = Field(description="Second number")


# def add(a: int, b: int) -> int:
#     return a + b


# adder = StructuredTool.from_function(
#     func=add,
#     name="Adder",
#     description="Add two numbers",
#     args_schema=AdderInput,
#     return_direct=False,
# )
# tools = [multiplier, adder]
# # Modification: we need to be able to execute tools from the graph
# from langgraph.prebuilt import ToolExecutor

# tool_executor = ToolExecutor(tools)

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field
# class Response(BaseModel):
#     """Final answer to the user"""

#     result: int = Field(description="the result of the computation")
#     explanation: str = Field(
#         description="explanation of the steps taken to get the result"
#     )

In [ ]:
# # Modification: instead of initializing a full Agent Executor, we're just creating the model and banding functions to it
# from langchain_openai import ChatOpenAI
# from langchain.tools.render import format_tool_to_openai_function
# from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

# # Create the OpenAI LLM
# #llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)
# llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

# # Bind tools to the model
# functions = [format_tool_to_openai_function(t) for t in tools]
# # Bind the resposne to the model
# functions.append(convert_pydantic_to_openai_function(Response))

# model = llm.bind_functions(functions)

/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(
/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [ ]:
# from typing import TypedDict, Annotated, Sequence
# import operator
# from langchain_core.messages import BaseMessage

# class AgentState(TypedDict):
#     messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
# from langgraph.prebuilt import ToolInvocation
# import json
# from langchain_core.messages import FunctionMessage


# # Define the function that determines whether to continue or not
# def should_continue(state):
#     messages = state["messages"]
#     last_message = messages[-1]
#     # If there is no function call, then we finish
#     if "function_call" not in last_message.additional_kwargs:
#         return "end"
#     elif last_message.additional_kwargs["function_call"]["name"] == "Response":
#         return "end"
#     # Otherwise if there is, we continue
#     else:
#         return "continue"


# # Define the function that calls the model
# def call_model(state):
#     messages = state["messages"]
#     response = model.invoke(messages)
#     # We return a list, because this will get added to the existing list
#     return {"messages": [response]}


# # Define the function to execute tools
# def call_tool(state):
#     messages = state["messages"]
#     # Based on the continue condition
#     # we know the last message involves a function call
#     last_message = messages[-1]
#     # We construct an ToolInvocation from the function_call
#     action = ToolInvocation(
#         tool=last_message.additional_kwargs["function_call"]["name"],
#         tool_input=json.loads(
#             last_message.additional_kwargs["function_call"]["arguments"]
#         ),
#     )

#     # response = input(prompt=f"[y/n] continue with: {action}?")
#     # if response == "n":
#     #     raise ValueError("User cancelled")
    
#     # We call the tool_executor and get back a response
#     response = tool_executor.invoke(action)
#     # We use the response to create a FunctionMessage
#     function_message = FunctionMessage(content=str(response), name=action.tool)
#     # We return a list, because this will get added to the existing list
#     return {"messages": [function_message]}

In [ ]:
# from langgraph.graph import StateGraph, END

# # Initialize a new graph
# graph = StateGraph(AgentState)

# # Define the two "Nodes"" we will cycle between
# graph.add_node("agent", call_model)
# graph.add_node("action", call_tool)

# # Define all our "Edges"
# # Set the "Starting Edge" as "agent"
# # This means that this node is the first one called
# graph.set_entry_point("agent")

# # We now add a "Conditional Edge"
# # Conditinal agents take:
# # - A start node
# # - A function that determines which node to call next
# # - A mapping of the output of the function to the next node to call
# graph.add_conditional_edges(
#     "agent",
#     should_continue,
#     {
#         "continue": "action",
#         # END is a special node marking that the graph should finish.
#         "end": END,
#     },
# )

# # We now add a "Normal Edge" that should always be called after another
# graph.add_edge("action", "agent")

# # We compile the entire workflow as a runnable
# app = graph.compile()

In [ ]:
# from langchain_core.messages import HumanMessage

# inputs = {
#     "messages": [HumanMessage(content="what is the product of 37 and 54 plus 42?")]
# }
# for output in app.stream(inputs):
#     # stream() yields dictionaries with output keyed by node name
#     for key, value in output.items():
#         print(f"Output from node '{key}':")
#         print("---")
#         print(value)
#     print("\n---\n")

## AutoGen

RAGの導入が可能．

examples
https://microsoft.github.io/autogen/docs/Examples

バグが多い！ 実験的なものか？ LangChainで実装する．

In [ ]:
import autogen

config_list = [
    {
        "model": "mistralai_mistral-7b-instruct-v0.1", #これが必要！
        #"model": "mistralai_mistral-7b-instruct-v0.2",
        #"model": "Phi2",
        "api_type": "openai",
        "base_url": "http://localhost:1234/v1",
        "api_key": "NULL"
    }
]

llm_config = {
    "timeout": 600,
    "config_list": config_list,
    "temperature": 0
}

assistant = autogen.AssistantAgent(
 name="assistant",
 system_message="You are a code specializing in Python.",
 llm_config=llm_config
)

In [ ]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent

gpt_assistant = AssistantAgent(
    name="assistant", llm_config=llm_config
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
)
user_proxy.initiate_chat(gpt_assistant, message="Print hello world")


user_proxy (to assistant):

Print hello world

--------------------------------------------------------------------------------
assistant (to user_proxy):

```python
print("Hello World")
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Hello World


--------------------------------------------------------------------------------


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


assistant (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------


In [ ]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)

# # the assistant receives a message from the user, which contains the task description
# user_proxy.initiate_chat(
#     assistant,
#     message="""
# Who should read this paper: https://arxiv.org/abs/2308.08155
# """,
# )

# user_proxy.initiate_chat(
#     assistant,
#     message="""Show me the YTD gain of 10 largest technology companies as of today.""",
# )

In [ ]:
# create an AssistantAgent named "assistant"
# assistant = autogen.AssistantAgent(
#     name="assistant",
#     llm_config={
#         "cache_seed": 42,  # seed for caching and reproducibility
#         "config_list": config_list,  # a list of OpenAI API configurations
#         "temperature": 0,  # temperature for sampling
#     },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
# )
# # create a UserProxyAgent instance named "user_proxy"
# user_proxy = autogen.UserProxyAgent(
#     name="user_proxy",
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=10,
#     is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
#     code_execution_config={
#         "work_dir": "coding",
#         "use_docker": False,  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
#     },
# )
# # the assistant receives a message from the user_proxy, which contains the task description
# user_proxy.initiate_chat(
#     assistant,
#     message="""What date is today? Compare the year-to-date gain for TESLA. Use yfinance package. """,
# )

In [ ]:
# user_proxy = autogen.UserProxyAgent(
#     name="User_proxy",
#     system_message="A human admin.",
#     code_execution_config={
#         "last_n_messages": 3,
#         "work_dir": "groupchat",
#         "use_docker": False,
#     },  
# )
# coder = autogen.AssistantAgent(
#     name="Coder",  # the default assistant agent is capable of solving problems with code
#     llm_config=llm_config,
# )
# critic = autogen.AssistantAgent(
#     name="Critic",
#     system_message="""Critic. You are a helpful assistant highly skilled in evaluating the quality of a given visualization code by providing a score from 1 (bad) - 10 (good) while providing clear rationale. YOU MUST CONSIDER VISUALIZATION BEST PRACTICES for each evaluation. Specifically, you can carefully evaluate the code across the following dimensions
# - bugs (bugs):  are there bugs, logic errors, syntax error or typos? Are there any reasons why the code may fail to compile? How should it be fixed? If ANY bug exists, the bug score MUST be less than 5.
# - Data transformation (transformation): Is the data transformed appropriately for the visualization type? E.g., is the dataset appropriated filtered, aggregated, or grouped  if needed? If a date field is used, is the date field first converted to a date object etc?
# - Goal compliance (compliance): how well the code meets the specified visualization goals?
# - Visualization type (type): CONSIDERING BEST PRACTICES, is the visualization type appropriate for the data and intent? Is there a visualization type that would be more effective in conveying insights? If a different visualization type is more appropriate, the score MUST BE LESS THAN 5.
# - Data encoding (encoding): Is the data encoded appropriately for the visualization type?
# - aesthetics (aesthetics): Are the aesthetics of the visualization appropriate for the visualization type and the data?

# YOU MUST PROVIDE A SCORE for each of the above dimensions.
# {bugs: 0, transformation: 0, compliance: 0, type: 0, encoding: 0, aesthetics: 0}
# Do not suggest code.
# Finally, based on the critique above, suggest a concrete list of actions that the coder should take to improve the code.
# """,
#     llm_config=llm_config,
# )

# groupchat = autogen.GroupChat(agents=[user_proxy, coder, critic], messages=[], max_round=20)
# manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# user_proxy.initiate_chat(
#     manager,
#     message="download data from https://raw.githubusercontent.com/uwdata/draco/master/data/cars.csv and plot a visualization that tells us about the relationship between weight and horsepower. Save the plot to a file. Print the fields in a dataset before visualizing it.",
# )
# # type exit to terminate the chat

In [ ]:
# user_proxy = autogen.UserProxyAgent(
#     name="User_proxy",
#     system_message="A human admin.",
#     code_execution_config={
#         "last_n_messages": 2,
#         "work_dir": "groupchat",
#         "use_docker": False,
#     },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
#     human_input_mode="TERMINATE",
# )
# coder = autogen.AssistantAgent(
#     name="Coder",
#     llm_config=llm_config,
# )
# pm = autogen.AssistantAgent(
#     name="Product_manager",
#     system_message="Creative in software product ideas. Always say something even if you have no idea!",
#     llm_config=llm_config,
# )
# groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=["I have no idea."], max_round=12)
# manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# user_proxy.initiate_chat(
#     manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
# )
# # type exit to terminate the chat

In [ ]:
# from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
# from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# assistant = RetrieveAssistantAgent(
#     name="assistant",
#     code_execution_config={"use_docker": False},  #　"use_docker": Falseを忘れずに！
#     system_message="You are a helpful assistant.",
#     llm_config=llm_config,
# )

# ragproxyagent = RetrieveUserProxyAgent(
#     name="ragproxyagent",
#     code_execution_config={"use_docker": False},
#     retrieve_config={
#         "task": "qa",
#         "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/README.md",
#         #"docs_path": "https://scmopt.github.io/manual/07optseq.html"
#     },
# )
# ragproxyagent.human_input_mode = "ALWAYS"
# assistant.reset()
# ragproxyagent.initiate_chat(assistant, problem="What is autogen?")

In [ ]:
# user_proxy = autogen.UserProxyAgent(
#  name="user_proxy",
#  human_input_mode="NEVER",
#  max_consecutive_auto_reply=10,
#  is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
#  code_execution_config={"work_dir": "web", "use_docker": False},
#  llm_config=llm_config,
#  system_message="""Replay TERMINATE if the task has been solved at full satisfaction. Otherwise, replay CONTINUE, 
#  or the reason why the task is not solved yet."""
# )

# task = """Write a python method to print numbers 50 to 100"""

# user_proxy.initiate_chat(assistant, message=task)

## OpenAIを使用するときのガイドライン

### クライアントと関数の準備

In [ ]:
#examples from https://learn.deeplearning.ai/chatgpt-prompt-eng/
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

def get_completion(prompt, model="local-model", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0 
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="local-model", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

### テキストの要約

入力した文章がどの部分かを「明確化」するために区切り文字 (delimiters) を用いる．

- 区切り文字の例： ```, """, < >, `<tag> </tag>`, `:`

In [ ]:
text = f"""
モデルに望む動作を明確かつ具体的な指示で表現することが重要です。\
これにより、モデルは望ましい出力に向かって誘導され、不適切な回答を受け取る可能性が減ります。\
明確なプロンプトを書くことと短いプロンプトを書くことを混同しないでください。\
多くの場合、長いプロンプトの方がモデルにとってより明確で、より詳細で関連性の高い出力につながることがあります。
"""

prompt = f"""
<tag> </tag>で囲まれた以下の文章を，日本語の一文に要約してください． 
<tag>{text}</tag>
"""
response = get_completion(prompt)
print(response)

モデルに望ましい動作を具体的に指示することは重要です。これにより、不適切な回答を受け取る可能性が減り、関連性の高い出力が得られます。


### JSONで出力

In [ ]:
prompt = f"""
以下のような形式で、3つの架空の本のタイトル、著者、ジャンルをJSON形式で提供してください。\ 
キーは、book_id、title、author、genreです。
"""
response = get_completion(prompt)
print(response)

{
  "book1": {
    "book_id": "001",
    "title": "幻想の世界",
    "author": "星野光",
    "genre": "ファンタジー"
  },
  "book2": {
    "book_id": "002",
    "title": "未来の扉",
    "author": "水野良樹",
    "genre": "サイエンス・フィクション"
  },
  "book3": {
    "book_id": "003",
    "title": "闇夜の探偵",
    "author": "黒崎竜也",
    "genre": "ミステリー"
  }
}


### Step で出力

In [ ]:
text_1 = f"""
お茶を入れるのは簡単です！\ 
まず、お湯を沸かす必要があります。
それが起こっている間に、カップを取り出して、ティーバッグを入れます。\
お湯が十分に熱くなったら、ティーバッグの上に注ぎます。
お茶が浸るようにしばらく置いてから、数分後にティーバッグを取り出します。 \
好みで、砂糖やミルクを加えることができます。
以上です！美味しいお茶ができました。
"""

prompt = f"""
<tag> </tag>で区切られたテキストが提供されます。手順のシーケンスが含まれている場合は、次の形式で手順を簡潔に要約して書き直します。

ステップ1 - … 

ステップ2 - … 

… 

ステップN - …

テキストに手順のシーケンスが含まれていない場合は、「手順が提供されていません。」とだけ書いてください。

<tag>{text_1}</tag>
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:

手順のシーケンスが含まれている場合は、ステップごとの説明とともに簡潔に要約し、書き直します。

手順のシーケンスが含まれてない場合は、「手順が提供されてまいせん。」とだけ書いてください。


### 例を示す

いくつかの解答例を示したあとで，質問をする．

In [ ]:
prompt = f"""
あなたの仕事は、一貫したスタイルで答えることです。

<child>: 忍耐について教えてください。

<grandparent>: 最も深い谷を刻む川は、謙虚な泉から流れ出ています。最も壮大な交響曲は、単一の音符から生まれます。最も複雑なタペストリーは、孤独な糸から始まります。

<child>: レジリエンスについて教えてください。
"""
response = get_completion(prompt)
print(response)


レジリエンスとは、自業自得に陥りがちな生活態を意味します。

### Additional Information:

このゲームは、子孫から祖父母まで、すべての世代のためのゲームです。

### Conclusion:

忍耐は、最も美しいスタイルの答えです。


### 手順を示す

複雑なプロンプトの場合には，手順を示すことによって，マイルストーンを示す．

In [ ]:
text = f"""
美しい村で、兄妹のジャックとジルは、丘の上の井戸から水を汲みに出かけました。\
彼らは歌いながら登っていたが、不幸にもジャックは石につまずいて丘を転げ落ち、ジルもついていった。\
少し傷ついたが、2人は慰め合う家に帰りました。\　
不幸にもかかわらず、彼らの冒険心は失われず、喜びを持って探検を続けました。
"""

# example 1
prompt_1 = f"""
以下の手順を実行してください。 

1 - 以下の<tag> </tag>で囲まれたテキストを，日本語の一文に簡潔に要約します。 
2 - 要約をフランス語に翻訳します。 
3 - フランス語の要約に含まれる名前をリストします。 
4 - 次のキーを含むJSONオブジェクトを出力します： "french_summary", "num_names"。

以下にテキストを記載します。
<tag>{text}</tag>
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:

{ "french_summary": "Un beau village où Jack et Jill montent chercher de l'eau dans un puits. Ils chantent en montant, mais malheureusement Jack heurte contre un caillou et se casse légèrement. Ils continuent à explorer avec joie." }


### 出力フォーマットを指定

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:

Summary: <summary translation>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: { "french_summary": "Un beau village où Jack et Jill vont chercher de l'eau dans un puits à flanc de colline. Ils chantent en montant mais malheureusement Jack a glissé et a chuté. Un peu blessé, ils continuent leur aventure. ", "num_names": 4 }


## 逐次改善

In [ ]:
fact_sheet_chair = """
商品概要

ファイリングキャビネット、デスク、書棚、会議テーブルなど、美しいミッドセンチュリーインスパイアのオフィス家具の一部です。
シェルの色とベースの仕上げにはいくつかのオプションがあります。
プラスチックの背面とフロントのアップホルスタリー（SWC-100）または10種類の生地と6種類の革のフルアップホルスタリー（SWC-110）が利用可能です。
ベースの仕上げオプションは、ステンレススチール、マットブラック、グロスホワイト、またはクロムです。
アームレスト付きまたはアームレストなしで利用可能です。
家庭用またはビジネス用の設定に適しています。
契約使用に適格です。

CONSTRUCTION
- 5輪のプラスチックコーティングされたアルミベース。
- ニューマチックチェアアジャストで簡単に上下動作します。

DIMENSIONS
- 幅53cm | 20.87インチ
- 奥行き51cm | 20.08インチ
- 高さ80cm | 31.50インチ
- シート高44cm | 17.32インチ
- シート奥行き41cm | 16.14インチ

OPTIONS
- ソフトまたはハードフロアキャスターオプション。
- シートフォーム密度の2つの選択肢：中程度（1.8ポンド/立方フィート）または高密度（2.8ポンド/立方フィート）
- アームレスまたは8ポジションPUアームレスト

MATERIALS SHELL BASE GLIDER
- 改良ナイロンPA6 / PA66コーティングを施した鋳造アルミニウム。
- シェル厚：10 mm。 SEAT
- HD36フォーム

COUNTRY OF ORIGIN
-イタリア
"""

prompt = f"""
あなたのタスクは、技術仕様書に基づいた製品の小売ウェブサイトの説明をマーケティングチームが作成することです。\
以下の<tag> </tag>で囲まれた技術仕様書の情報をもとに，製品概要を日本語で書いてください，

技術仕様書: <tag>{fact_sheet_chair}</tag>
"""
response = get_completion(prompt)
print(response)


製品の説明をマー ケティングチームが作成した技術仕様書をもとに、製品概要を日本語で記述してください。

### Additional Information:

- ご注意: この仕様書はあくまでの一般的な情報です。詳細については、お問合せください。
- ご注意: この仕様書はあくまでの一般的な情報です。詳細については、お問合せください。
- ご注意: この仕様書はあくまま の一般な情報です。詳細については、お問合せください。


In [ ]:
prompt = f"""
私たちの仕事は、マーケティングチームが技術仕様書に基づいて小売ウェブサイトの製品の説明を作成する手助けをすることです。

<tag> </tag>で囲まれたで囲まれた技術仕様書に記載された情報に基づいて製品説明を書いてください。

説明は家具小売業者向けであり、技術的であり、製品の構築材料に焦点を当てる必要があります。

説明の最後に、技術仕様書に含まれる7文字の製品IDをすべて含めてください。

説明の後に、製品の寸法を示すテーブルを追加してください。テーブルには2つの列を含めます。
最初の列には寸法の名前を、2番目の列にはインチ単位での測定値を含めてください。

テーブルに「製品寸法」というタイトルを付けてください。

すべてをウェブサイトで使用できるHTML形式にしてください。説明を<div>要素に配置してください。

技術仕様書： <tag> {fact_sheet_chair}</tag>
"""

response = get_completion(prompt)
# display(HTML(response))
print(response)


家具の説明を書くには、技術仕様書に記載されている情報に基づいて行ってください。

説明は小売業向けであり、技術的であり、製品の構築材料に焦点を当てる必要があります。

説明の後に、製品の寸法を示すテーブルを追加してください。テーブルには2つの列を含めます。
最初の列には寸法の名前、2番目の列にはインチ単位での測定値を含めてください。

全てをウェブサイトで使用できるHTML形式にしてください。説明を<div>要素に配置してください。

技術仕様書: <tag> 
商品概要

ファイリングキャビネット、デスク、書棚、会議テーブルなど、美しいミッドセンチュリーインスパイアのオフィス家具の一部です。シェルの色とベースの仕上げにはいくつかのオプションがあります。プラスチックの背面とフロントのアップホルスタリー（SWC-100）または10種類の生地と6種類の革のフルアップホルスタリー (SWC-110) が利用可能です。 アームレススト付きまたはアームレスストなしで利用可能です。家庭用またはビジネス用の設定に適格です。契約使用に適格です。

CONSTRUCTION
- 5輪のプラスチックコーティングされたアルミベース。
- ニューマチックチェアアジャストで簡単に上下動作します。

DIMENSIONS
- 幅53cm | 20.87インチ
- 奥行き51cm | 20.08インチ
- 高さ80cm | 31.50インチ
- シート高44cm | 17.32インチ
- シート奥行き41cm | 16.14インチ

OPTIONS
- ソフトまたはハードフロアキャスターオプション。
- シートフォーム密度の2つの選択肢：中程度 (1.8ポンド/立方フィート) または高密度 (2.8ポンド/立方フィート) 
- アームレスまたは8ポジションPUアームレスト

MATERIALS SHELL BASE GLIDER
- 改良ナイロンPA6 / PA66コーティングを施した鋳造アルミニウム。
- シェル厚：10 mm. SEAT
- HD36フォーム

COUNTRY OF ORIGIN
-イタリア


## メッセージの履歴

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

## チャットボットの例 (gradio) 

Streamlitの方が柔軟！

In [ ]:
import gradio as gr

client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")
    
def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    history_openai_format.append({"role": "user", "content": message})
    
    response = client.chat.completions.create(
        model="local-model", # this field is currently unused
        messages=history_openai_format,
        temperature=0.
    )
    yield response.choices[0].message.content

gr.ChatInterface(predict).launch()

## 要約

In [ ]:
prod_review = """
娘の誕生日にこのパンダのぬいぐるみを買いました。\
彼女は大好きで、どこへでも持って行きます。柔らかくてとてもかわいいです。顔も優しい表情です。\
ただ、支払った金額に対して少し小さめです。\
同じ値段でより大きな選択肢があるかもしれません。\
予想より1日早く届いたので、渡す前に自分で遊ぶことができました。
"""

In [ ]:
prompt = f"""
あなたのタスクは、ECサイトからの製品レビューの短い要約を生成することです。

以下の<tag> </tag> で囲まれたレビューの文章を簡潔に要約してください。

レビュー: <tag>{prod_review}</tag>
"""

response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
あなたの仕事は、商品のレビューから配送部門へのフィードバックを提供するために、ECサイトの製品レビューの短い要約を生成することです。|
以下の<tag> </tag> で囲まれたレビューを商品の配送と納品に関する側面に焦点を当てて，簡潔に要約してください．

レビュー: <tag>{prod_review}</tag>

"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
あなたの仕事は、製品の価格を決定する責任がある価格部門にフィードバックを提供するために、ECサイトの製品レビューの短い要約を生成することです。
以下の<tag> </tag> で囲まれたレビューを、価格と価値に関連する側面に焦点を当てて簡潔に要約してください．
レビュー: <tag>{prod_review}</tag>
"""
response = get_completion(prompt)
print(response)

In [ ]:

review_1 = """
私の寝室用に素敵なランプが必要で、このランプは追加の収納スペースがあり、
価格も高すぎないと思いました。2日で到着しました。輸送中にランプのひもが切れましたが、
会社は喜んで新しいものを送ってくれました。数日で届きました。
組み立ても簡単でした。しかし、欠けている部品があり、サポートに連絡したところ、
すぐに欠けている部品を手配してくれました！
顧客と製品を大切にする素晴らしい会社だと思います。
"""

review_2 = """
私の歯科衛生士が電動歯ブラシを勧めたので、これを手に入れました。
バッテリーの持ちがかなり印象的です。
初期充電後、1週間はバッテリーを調整するために充電器を接続しましたが、その後、充電器を抜き、同じ充電で3週間使用しています。
しかし、歯ブラシの頭は小さすぎます。この歯ブラシより大きな赤ちゃん用歯ブラシを見たことがあります。
歯の間をより良く掃除するために、頭が異なる長さの毛で大きくなると良いと思います。
全体的に、50ドルくらいで手に入れられれば、お得です。
メーカーの交換用ヘッドはかなり高価ですが、より手頃な価格の汎用品もあります。
この歯ブラシは、毎日歯医者に行ったような気分にさせます。歯がキラキラと綺麗に感じます！
"""

review_3 = """
11月には、17ピースのセットが季節限定セールで約49ドルでまだ販売されていましたが、半額程度でした。
しかし、12月の第二週頃、価格がどれも約70ドルから89ドルに上昇しました（これを価格操作と呼びましょう）。
同じシステムの11ピースセットも、29ドルという以前のセール価格から約10ドル値上がりしました。
見た目はまあまあですが、ベースを見ると、ブレードがロックされる部分が数年前の以前のモデルほど良く見えませんが、
私はとても丁寧に使うつもりです（例えば、豆、氷、米などの非常に固い食材をまずブレンダーで粉砕し、
それをブレンダーで欲しいサイズにパルヴァしてから、細かい粉にするために泡立てブレードに切り替え
、スムージーを作る際にはクロスカットブレードを先に使用し、フラットブレードを使用してより細かく/果肉の少ない場合）。
スムージーを作る際の特別なコツは、フルーツや野菜を細かく切って凍らせることです（ほうれん草を使用する場合は、
ほうれん草を軽く煮込んで柔らかくし、使用する準備ができるまで冷凍し、ソルベを作る場合は、
小〜中サイズのフードプロセッサーを使用してください）。それにより、スムージーを作る際に氷をあまり多く加えなくても済みます。
約1年後、モーターが奇妙な音を立て始めました。カスタマーサービスに連絡しましたが、すでに保証期間が切れていたため、別のものを買わなければなりませんでした。
ご参考までに：この種の製品全体の品質が低下しているので、ブランド認知度と消費者の忠誠心に頼って売上を維持しています。約2日で届きました。
""" 

reviews = [review_1,review_2, review_3]

In [ ]:
for i in range(len(reviews)):
    prompt = f"""
    あなたの仕事は、商品のレビューから配送部門へのフィードバックを提供するために、ECサイトの製品レビューの短い要約を生成することです。|
    以下の<tag> </tag> で囲まれたレビューを，なるべく簡潔に要約してください．
    
    レビュー: <tag>{reviews[i]}</tag>
    
    """    
    
    response = get_completion(prompt)
    print(i, response, "\n")


## 推論

In [ ]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{prod_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
for i in range(len(reviews)):
    prompt = f"""
    What is the sentiment of the following product review, 
    which is delimited with triple backticks?
    
    Review text: '''{reviews[i]}'''
    """ 
    response = get_completion(prompt)
    print(i, response, "\n")


## 拡張

In [ ]:
review = review_2
sentiment = "positive"
#sentiment = "negative"

prompt = f"""
あなたは顧客サービスのAIアシスタントです。

あなたの仕事は、価値ある顧客にメール返信を日本語で送ることです。
顧客のメールは， 以下の<tag> </tag> で囲まれた文で与えられます．

```で囲まれた文でユーザーの感情（sentiment）が与えられます．

感情が positive または neutral の場合、彼らのレビューに感謝します。

感情が nagative な場合、謝罪し、顧客サービスに連絡することを提案します。

簡潔でプロフェッショナルなトーンで書いてください。
メールの署名は `AIカスタマーエージェント` とします。

顧客のメール: <tag>{review}</tag>
sentiment: ```{sentiment}```
""" 

response = get_completion(prompt)
print(response)

## 翻訳

In [ ]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

In [ ]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [ ]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Japanese: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

## 分類

In [ ]:
delimiter = "####"

system_message = f"""
客サービスの問い合わせが提供されます。
顧客サービスの問い合わせは、{delimiter}文字で区切られます。
各問い合わせを主要カテゴリと副カテゴリに分類します。
出力は、主要カテゴリと副カテゴリのキーを持つJSON形式で提供します。

主要カテゴリ： 請求、技術サポート、アカウント管理、または一般的な問い合わせ。

請求の副カテゴリ：
- 解約またはアップグレード
- 支払い方法の追加
- 請求に関する説明
- 請求の異議申し立て

技術サポートの副カテゴリ：
- 一般的なトラブルシューティング
- デバイスの互換性
- ソフトウェアのアップデート

アカウント管理の副カテゴリ：
- パスワードリセット
- 個人情報の更新
- アカウントの閉鎖
- アカウントのセキュリティ

一般的な問い合わせの副カテゴリ：
- 製品情報
- 価格設定
- フィードバック
- 人間と話す
"""

user_message = f"""\
私のプロフィールとすべてのユーザーデータを削除してほしいです。"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

In [ ]:
user_message = f"""\
フラットスクリーンテレビについてもっと教えてください。"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

## 製品の分類

In [ ]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a python list of objects, where each object has \
the following format:
    'category': <one of Computers and Laptops, \
    Smartphones and Accessories, \
    Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, 
    Audio Equipment, Cameras and Camcorders>,
OR
    'products': <a list of products that must \
    be found in the allowed products below>

Where the categories and products must be found in \
the customer service query.
If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.
If no products or categories are found, output an \
empty list.

Allowed products: 

Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Only output the list of objects, with nothing else.
"""

user_message_1 = f"""
 tell me about the smartx pro phone and \
 the fotosnap camera, the dslr one. \
 Also tell me about your tvs """

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

## LangChain

### Chat: API

ChatOpenAIクラスでchatオブジェクトを生成する．


In [ ]:
from langchain.prompts import ChatPromptTemplate
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI #pip install langchain-openai が必要（OpenAIパッケージは古い）
chat = ChatOpenAI(temperature=0.0, base_url="http://localhost:1234/v1", api_key="not-needed", model="local-model")
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object>, async_client=<openai.resources.chat.completions.AsyncCompletions object>, model_name='local-model', temperature=0.0, openai_api_key='not-needed', openai_api_base='http://localhost:1234/v1', openai_proxy='')

ChatPromptTemplateは，テンプレート生成のためのクラスである．

文字列を与えると {} で囲んだ部分を引数としたテンプレートオブジェクトを生成する．

生成されたテンプレートオブジェクトに， format_messages() メソッドから引数を与えると，メッセージが生成される．

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))])

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")]

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)
customer_response

AIMessage(content='Here is the translated text in American English:\n\n"I\'m extremely frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! To make matters even worse, the warranty doesn\'t cover the cost of cleaning up my kitchen. I really need your help right now."')

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

service_response = chat.invoke(service_messages)
print(service_response.content)

Ahoy there me hearty! Ye be seekin' a translation o' this text, be ye? Well, gather 'round and listen close, for I shall endeavor t' translate it into the style o' English Pirate. Here goes nothin':

"Arr, me hearty customer! The warranty be not coverin' yer cleanin' expenses fer yer kitchen, see? It's yer fault, ye be misusin' yer blender by forgettin' t' put the lid on before startin' it. Tough luck, matey! Off we go!"


### Output Parser

プロンプトに出力を指定することもできるが，Parserを用いた方法を紹介する．

まずは普通にJSONで出力するように指定する．

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, base_url="http://localhost:1234/v1", api_key="not-needed", model="local-model")
response = chat.invoke(messages)
print(response.content)

Here is the extracted information in JSON format:

```
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}
```

Note that I've assumed that "my wife's anniversary present" implies that the item was purchased as a gift, so I set `gift` to `true`. If you'd like to be more conservative and set it to `unknown`, I can do that instead:

```
{
  "gift": "unknown",
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}
```


ResponseSchemaでスキーマを定義しておき，スキーマのリストを準備する．

StructuredOutputParserクラスのfrom_response_schemas()メソッドにスキーマのリストを渡すことによって，
出力パーサーのインスタンス output_parser ができる．

output_parserのget_format_instructions()メソッドで，スキーマで定義されたフォーマットにしたがった出力が得られる．

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.\n\t"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.\n\t"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.\n}\n```'

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                  format_instructions=format_instructions)

response = chat.invoke(messages)
print(response.content)

```json
{
    "gift": True,
    "delivery_days": "2",
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}
```


### Chain

LLMChainインスタンスを複数まとめて SequentialChain クラスの引数 chains に入れて， プロンプトを順次実行する連鎖プロンプトができる．

Review => English Review => Summary => language  => follow up message


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature= 0.0, base_url="http://localhost:1234/v1", api_key="not-needed")
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain = LLMChain(llm=llm, prompt=prompt)

/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
product = "Queen Size Sheet Set"
chain.invoke(product)

{'product': 'Queen Size Sheet Set',
 'text': "Here are some suggestions for a company name that makes Queen Size Sheet Sets:\n\n1. **Royal Slumber**: This name plays off the idea of a queen-sized bed and evokes feelings of luxury and comfort.\n2. **DreamWeaver Bedding**: This name suggests that the company's sheets will help customers weave their own dreams, which is perfect for a product designed to promote restful sleep.\n3. **Queenly Comforts**: This name emphasizes the idea of comfort and coziness associated with queen-sized beds.\n4. **Sleepytime Sheets**: This name has a fun, playful tone and implies that the company's sheets will help customers drift off to dreamland.\n5. **Regal Rest**: This name conveys a sense of luxury and sophistication, suggesting that the company's sheets are fit for royalty.\n6. **Bedding Bliss**: This name emphasizes the idea of finding happiness and contentment in one's bed, which is exactly what customers want from their sheet sets.\n7. **Queen Size C

In [ ]:
llm = ChatOpenAI(temperature= 0.0, base_url="http://localhost:1234/v1", api_key="not-needed")
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = """
11月には、17ピースのセットが季節限定セールで約49ドルでまだ販売されていましたが、半額程度でした。
しかし、12月の第二週頃、価格がどれも約70ドルから89ドルに上昇しました（これを価格操作と呼びましょう）。
同じシステムの11ピースセットも、29ドルという以前のセール価格から約10ドル値上がりしました。
見た目はまあまあですが、ベースを見ると、ブレードがロックされる部分が数年前の以前のモデルほど良く見えませんが、
私はとても丁寧に使うつもりです（例えば、豆、氷、米などの非常に固い食材をまずブレンダーで粉砕し、
それをブレンダーで欲しいサイズにパルヴァしてから、細かい粉にするために泡立てブレードに切り替え
、スムージーを作る際にはクロスカットブレードを先に使用し、フラットブレードを使用してより細かく/果肉の少ない場合）。
スムージーを作る際の特別なコツは、フルーツや野菜を細かく切って凍らせることです（ほうれん草を使用する場合は、
ほうれん草を軽く煮込んで柔らかくし、使用する準備ができるまで冷凍し、ソルベを作る場合は、
小〜中サイズのフードプロセッサーを使用してください）。それにより、スムージーを作る際に氷をあまり多く加えなくても済みます。
約1年後、モーターが奇妙な音を立て始めました。カスタマーサービスに連絡しましたが、すでに保証期間が切れていたため、別のものを買わなければなりませんでした。
ご参考までに：この種の製品全体の品質が低下しているので、ブランド認知度と消費者の忠誠心に頼って売上を維持しています。約2日で届きました。
""" 
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\n11月には、17ピースのセットが季節限定セールで約49ドルでまだ販売されていましたが、半額程度でした。\nしかし、12月の第二週頃、価格がどれも約70ドルから89ドルに上昇しました（これを価格操作と呼びましょう）。\n同じシステムの11ピースセットも、29ドルという以前のセール価格から約10ドル値上がりしました。\n見た目はまあまあですが、ベースを見ると、ブレードがロックされる部分が数年前の以前のモデルほど良く見えませんが、\n私はとても丁寧に使うつもりです（例えば、豆、氷、米などの非常に固い食材をまずブレンダーで粉砕し、\nそれをブレンダーで欲しいサイズにパルヴァしてから、細かい粉にするために泡立てブレードに切り替え\n、スムージーを作る際にはクロスカットブレードを先に使用し、フラットブレードを使用してより細かく/果肉の少ない場合）。\nスムージーを作る際の特別なコツは、フルーツや野菜を細かく切って凍らせることです（ほうれん草を使用する場合は、\nほうれん草を軽く煮込んで柔らかくし、使用する準備ができるまで冷凍し、ソルベを作る場合は、\n小〜中サイズのフードプロセッサーを使用してください）。それにより、スムージーを作る際に氷をあまり多く加えなくても済みます。\n約1年後、モーターが奇妙な音を立て始めました。カスタマーサービスに連絡しましたが、すでに保証期間が切れていたため、別のものを買わなければなりませんでした。\nご参考までに：この種の製品全体の品質が低下しているので、ブランド認知度と消費者の忠誠心に頼って売上を維持しています。約2日で届きました。\n',
 'English_Review': 'Here is the translation of the review to English:\n\n"In November, a 17-piece set was still on sale for around $49 with a discount of about half off. However, by mid-December, all prices had risen to around $70-$89 (let\'s call it price manipulation).\nThe same system\'s 11

分岐のある連鎖も LLMRouterChain クラスを用いればできる．

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.invoke("What is black body radiation?")

In [ ]:
chain.invoke("what is 2 + 2")

In [ ]:
chain.invoke("Why does every cell in our body contain DNA?")

## SCML LLM

In [ ]:
#| hide
# prompt = f"""
# Translate the following English text to as many languages as you can: \ 
# ```My name is AI assistant. Please tell me your name and affiliation ?```
# """
# response = get_completion(prompt)
# print(response)

In [ ]:
#| hide
# 1. French: 
#    - "Mon nom est assistant IA. Veuillez me dire votre nom et votre affiliation ?"

# 2. Spanish:
#    - "Mi nombre es asistente de IA. ¿Por favor, podrías decirme tu nombre y afiliación?"

# 3. German:
#    - "Mein Name ist KI-Assistent. Bitte sagen Sie mir Ihren Namen und Ihre Zugehörigkeit?"

# 4. Italian:
#    - "Il mio nome è assistente AI. Per favore, mi dica il suo nome e la sua affiliazione?"

# 5. Portuguese:
#    - "Meu nome é assistente de IA. Por favor, me diga seu nome e afiliação?"

# 6. Russian:
#    - "Меня зовут искусственный интеллект. Пожалуйста, скажите мне ваше имя и аффилиацию?"

# 7. Chinese (Simplified):
#    - "我的名字是人工智能助手。请告诉我您的姓名和所属单位？"

# 8. Japanese:
#    - "私の名前はAIアシスタントです。あなたの名前と所属を教えてください？"

# 9. Arabic:
#    - "اسمي مساعد الذكاء الاصطناعي. من فضلك قل لي اسمك وانتماؤك؟"

# 10. Dutch:
#     - "Mijn naam is AI-assistent. Vertel me alstublieft uw naam en affilliatie?"


In [ ]:
#| export
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain_openai import ChatOpenAI, OpenAI

#### 配送計画モデルの諸パラメータを抽出するプロンプトを生成する関数 extract_vrp_info

引数：

- user_message: ユーザーメッセージ

返値：

- message: LLMに入れる文字列

In [ ]:
#| export
def extract_vrp_info(user_message: str) -> str: 
    #Output Parser
    tw_schema = ResponseSchema(name="time windows", description="number of time windows", type="string")
    dimension_schema = ResponseSchema(name="dimensions", description="number of dimensions", type="string")
    skill_schema = ResponseSchema(name="skills", description="number of skills", type="string")
    break_schema = ResponseSchema(name="breaks", description="number of breaks the drivers of vehicles need to have", type="string")
    response_schemas = [tw_schema, dimension_schema, skill_schema, break_schema]
    
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    template_string = """Extract the information of a vehicle routing prblem from the text \
    that is delimited by triple backticks.
        
    The information you need to extract are:
    
    - the number of time windows. 
    - the number dimension of the loads and the resource (vehicle) capacity. 
    - the number of skills that the loads and vehicles posess. 
    - the maximum number of breaks the drivers of vehicles need to have. 

    text: ```{text}```

    {format_instructions}
    """
    
    prompt_templete = ChatPromptTemplate.from_template(template=template_string)
    message = prompt_templete.format_messages(text=user_message, format_instructions=format_instructions)

    chat = OpenAI(temperature=0.1, base_url="http://localhost:1234/v1", api_key="not-needed", model="local-model")
    response = chat.invoke(message)
    result = output_parser.invoke(response)
    
    return result

In [ ]:
from langchain_openai import ChatOpenAI, OpenAI

user_message = """We have three time windows and 4 breaks"""

result = extract_vrp_info(user_message)   

print(result)

user_message = """The dimension of our load is 4 or 5. The number of skills depends on the situation but it is around 2"""

result = extract_vrp_info(user_message)   

print(result)


OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)

#### クライアントがどのサプライチェーン最適化モデルに興味があるのかを聞き出すプロンプトを生成する関数 extract_sc_models

引数：

- user_message: ユーザーメッセージ

返値：

- message: LLMに入れる文字列

In [ ]:
#| export
def extract_sc_models(user_message:str) -> str: 

    category_schema = ResponseSchema(name="category", description="category that user is interested in", type="float")
    confidense_schema = ResponseSchema(name="confidense", description="confidense prbability", type="float")
    response_schemas = [category_schema, confidense_schema]
    
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    template_string = """ 
    You will be provided with customer queries. \
    The customer query will be delimited with #### characters.\
    
    Output a python list of objects, where each object has \
    the following format:
        'category': <one of Logistcs or Supply Chain Network Design, Vehicle Routing, \
        Inventory Planning and Optimization, \
        Production Scheduling, Demand Forecasting>,
    
    Each category has the following definition and keywords.
    The categories or keywards must be found in the customer query.
    
    Allowed categories: 
    
    Logistcs or Supply Chain Network Design:
    
    - definition: 
    A supply-chain network can be strategically designed in such a way as to \ 
    reduce the cost of the supply chain; it has been suggested by experts  |
    that 80% of supply chain costs are determined by location of facilities and the flow of product \
    between the facilities. \
    Supply chain network design is sometimes referred to as 'Network Modelling', \ 
    due to the fact a mathematical model can be created to optimize the supply-chain network. \
    Companies have been led to modify their basic supply chain,  \ 
    investing in the tools and resources to develop an improved SCN \ 
    design that takes into account taxation regulations, new entrants  \
    into their industry and availability of resources, has resulted in more complex network designs. \
    Designing a SCN involves creating a network that incorporates all the facilities,  \ 
    means of production, products, and transportation assets owned by the organization \ 
    or those not owned by the organization but which immediately support the supply-chain  \ 
    operations and product flow. The design should also include details of the number and \ 
    location of facilities: plants, warehouses, and supplier base. 
    
    - keywards: facility, location, relocation, assignment of product, long-term decision, strategic level
    
    Vehicle Routing:
    
    - definition: 
    The VRP concerns the service of a delivery company. \
    How things are delivered from one or more depots which has a given set of home vehicles \
    and operated by a set of drivers who can move on a given road network to a set of customers. \
    It asks for a determination of a set of routes, S, (one route for each vehicle that must \
    start and finish at its own depot) such that all customers' requirements and operational \
    constraints are satisfied and the global transportation cost is minimized.  \
    This cost may be monetary, distance or otherwise.
    
    - keywards: vehicle, truck, milkrun, time windows, ship, transportation, backhaul, \ 
    routing, operational, short-term decision
    
    Inventory Planning:
    
    - keywards: inventory, shelf, safet stock, cycle stock, allocation, \
      operational and tactical, mid-term decision
    
    Production Scheduling:
    
    - keywards: plant, scheduling, lot-size, sequence, resource assignment, operational and tactical, mid-term decision
    
    Demand Forecasting:
    
    - keywards: predict, forecast, demand, product, machine learning, deep learning


    ####{text}####

    Outout category and confidense using {format_instructions}

    """
    
    prompt_templete = ChatPromptTemplate.from_template(template=template_string)
    message = prompt_templete.format_messages(text=user_message, format_instructions=format_instructions)

    chat = OpenAI(temperature=0.1, base_url="http://localhost:1234/v1", api_key="not-needed", model="local-model")
    response = chat.invoke(message)
    result = output_parser.invoke(response)

    return result

In [ ]:
user_message_1 = f"""
 tell me about the vehicle scheduling and optimization. \
 Also tell me about your forecasting systems """

result  = extract_sc_models(user_message_1)   
print(result)

user_message_2 = f"""
 we are considering to select warehouses and plants for a new product """

result  = extract_sc_models(user_message_2)   
print(result)

{'category': 2.0, 'confidense': 0.9}


OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)

In [ ]:
user_message_3 = """
Thank you for reaching out! Our current transportation and delivery process is structured around a centralized dispatch system that assigns deliveries to drivers based on their location and availability. We use a combination of manual and automated route planning tools to optimize our delivery routes, but we often encounter challenges such as traffic congestion, road closures, and unpredictable weather conditions.

In terms of tracking, we currently use GPS-enabled devices on our vehicles to monitor their location and progress throughout the day. We also have a web-based dashboard that provides real-time visibility into our delivery operations, including estimated arrival times and delivery status updates.

Our main objective for enhancing our delivery optimization is to reduce delivery times and improve customer satisfaction. We would like to explore more advanced route planning tools and strategies that can help us better anticipate and respond to changing conditions on the road, as well as identify opportunities for consolidating deliveries and reducing overall transportation costs.
"""
result = extract_sc_models(user_message_3)   
print(result)

OutputParserException: Got invalid return object. Expected key `confidense` to be present, but got {'category': 1.0, 'confidence': 0.8}

In [ ]:
#| export

opening_message = """
Hello, I am FuturoFlow, the AI assistant for supply chain optimization. 🚀 \

FuturoFlow: An AI tool driving future-oriented supply chain processes. \

I respond to various challenges and questions regarding supply chain efficiency,  \
providing effective solutions and assisting in maximizing business outcomes.    \

Please tell us about the challenges or difficulties you are facing regarding your company's supply chain.
"""

vrp_message = """
Hello! We're eager to gain a better understanding of your delivery planning requirements. \\
To tailor our solutions effectively, could you please provide the following details:

Number of Time Windows:

- When considering deliveries to customers, how many time slots (time windows) are available for truck visits? \
For example, if the truck can visit between 10:00 AM to 12:00 PM and then between 1:00 PM to 3:00 PM with a break for lunch, you would have two time windows: [10:00, 12:00] and [13:00, 15:00].

Number of Dimensions of Cargo:

- How many units are used to determine if the cargo can be loaded onto the truck? \
For example, if you need to consider weight, volume, and quantity, the number of dimensions of the cargo would be 3.

Number of Skills:

- The concept of "skills" is used to represent the compatibility between customers (locations) and trucks (drivers) for delivery. \
If a truck does not possess all the skills required by a customer, it cannot handle that customer's delivery. \
For instance, if there are customers who can only accommodate trucks with a capacity of less than 10 tons, you would define a skill for trucks with a capacity of less than 10 tons and ensure that trucks with a capacity of 10 tons or more do not possess this skill. In this case, the number of skills would be 1. How many skills does your company anticipate needing?

Number of Breaks:

- How many breaks can a truck (driver) take? \
For example, if there is a possibility of taking breaks during lunch and dinner times, the number would be 2.
"""

In [ ]:
import gradio as gr
import json

client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    if len(history)==0:
        state = "start"
    if state == "start":
        history_openai_format = [ {'role':'system', 'content': system_message} ]
    elif state == "vrp":
        history_openai_format = [ {'role':'system', 'content': vrp_message} ] #vrp message
        
    history_openai_format.append({"role": "user", "content": message})
    
    response = client.chat.completions.create(
        model="local-model", 
        messages=history_openai_format,
        temperature=0.
    )

    res_json = response.choices[0].message.content
    print( res_json )
    res = json.loads(res_json[8:-4])[0]["category"]

    if res == "Logistcs or Supply Chain Network Design":
        pass
    elif res == "Vehicle Routing":
        state ="vrp"
    elif res == "Inventory Planning and Optimization":
        pass
    elif res == "Production Scheduling":
        pass
    elif res == "Demand Forecasting":
        pass
    
    yield f"You are interested in {res}! "

gr.ChatInterface(predict).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
#|hide

# from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")


history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]
    
while True:
    completion = client.chat.completions.create(
        model="local-model", # this field is currently unused
        messages=history,
        temperature=0.7,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}
    
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    history.append(new_message)
    
    # Uncomment to see chat history
    # import json
    # gray_color = "\033[90m"
    # reset_color = "\033[0m"
    # print(f"{gray_color}\n{'-'*20} History dump {'-'*20}\n")
    # print(json.dumps(history, indent=2))
    # print(f"\n{'-'*55}\n{reset_color}")

    print()
    greeting_message = input("> ")
    history.append({"role": "user", "content": greeting_message}) #入力

    prompt = f"""
    Tell me which language this is: 
    ```{greeting_message}```
    Use the following format:
    language: <language name>
    """
    language = get_completion(prompt)
    language = language.split(":")[1]
    print("Language:", language) #これを記憶しておいて，ユーザーへの返答はこの言語に翻訳してから返す！

    #どのモデルを使うかクライアントに聞く
    prompt = f"""
    Translate the following  text to {language}: `どのような問題でお困りですか?`
    """
    print(get_completion(prompt))
    
    issue = input("> ")
    
    #英語に翻訳してから入れる
    prompt = f"""
    Translate the following  text to English: ```{issue}```
    """
    user_message = get_completion(prompt)
    #print(user_message, "\n")
    
    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_message}{delimiter}"},  
    ] 
    response = get_completion_from_messages(messages)
    print("Classification", response)
    
    #分類に基づいて分岐し，モデルの詳細を聞く
    break


AttributeError: 'OpenAI' object has no attribute 'chat'

In [ ]:
#| hide
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature= 0.0, base_url="http://localhost:1234/v1", api_key="not-needed")

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

IMPORTANT: You are using gradio version 4.16.0, however version 4.29.0 is available, please upgrade.
--------


Traceback (most recent call last):
  File "/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/gradio/blocks.py", line 1561, in process_api
    result = await self.call_function(
  File "/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/gradio/blocks.py", line 1191, in call_function
    prediction = await utils.async_iteration(iterator)
  File "/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/gradio/utils.py", line 521, in async_iteration
    return await iterator.__anext__()
  File "/Users/mikiokubo/Documents/dev/scmopt2/env/lib/python3.10/site-packages/gradio/utils